# Data description

This is one of multiple ECoG datasets from Miller 2019, recorded in a clinical settings with a variety of tasks. Raw data and dataset paper are here:

https://exhibits.stanford.edu/data/catalog/zk881ps0522
https://www.nature.com/articles/s41562-019-0678-3

This particular dataset was originally described in this paper: 

*Miller, Kai J., Gerwin Schalk, Eberhard E. Fetz, Marcel Den Nijs, Jeffrey G. Ojemann, and Rajesh PN Rao. "Cortical activity during motor execution, motor imagery, and imagery-based online feedback." Proceedings of the National Academy of Sciences (2010): 200913697.*

`dat1` and `dat2` are data from the two blocks performed in each subject. The first one was the actual movements, the second one was motor imagery. For the movement task, from the original dataset instructions:

*Patients performed simple, repetitive, motor tasks of hand (synchronous flexion and extension of all fingers, i.e., clenching and releasing a fist at a self-paced rate of ~1-2 Hz) or tongue (opening of mouth with protrusion and retraction of the tongue, i.e., sticking the tongue in and out, also at ~1-2 Hz). These movements were performed in an interval-based manner, alternating between movement and rest, and the side of move- ment was always contralateral to the side of cortical grid placement.*

For the imagery task, from the original dataset instructions:

*Following the overt movement experiment, each subject performed an imagery task, imagining making identical movement rather than executing the movement. The imagery was kinesthetic rather than visual (“imagine yourself performing the actions like you just did”; i.e., “don’t imagine what it looked like, but imagine making the motions”).*

Sample rate is always 1000Hz, and the ECoG data has been notch-filtered at 60, 120, 180, 240 and 250Hz, followed by z-scoring across time and conversion to float16 to minimize size. Please convert back to float32 after loading the data in the notebook, to avoid unexpected behavior. 

Both experiments: 
* `dat['V']`: continuous voltage data (time by channels)
* `dat['srate']`: acquisition rate (1000 Hz). All stimulus times are in units of this.  
* `dat['t_on']`: time of stimulus onset in data samples
* `dat['t_off']`: time of stimulus offset, always 400 samples after `t_on`
* `dat['stim_id`]: identity of stimulus (11 = tongue, 12 = hand), real or imaginary stimulus
* `dat['scale_uv']`: scale factor to multiply the data values to get to microvolts (uV). 
* `dat['locs`]`: 3D electrode positions on the brain surface

# Project Proposal

## Distinguishing motor execution from motor imagery

Keywords: motor execution, motor imagery, ECoG, ECoG decoding, event related potential, power spectrum density, hand control, tongue control, motor cortex, Miller 2019, efficient real-time decoding. 

We propose to understand the neurological differences between actual movement and motor imagery. Furthermore, we would like to decode signals from the motor cortex to predict movement onset. To achieve this, we will leverage the ECoG dataset from Miller 2019 on motor imagery. 

Our research will focus in two questions: 
1. How is motor execution different from motor imagery? 
2. Can we decode hand movement and tongue movement? 

More precisely, for (1), we ask whether there exists a model that can differentiate between motor execution and motor imagery? Does it generalize across the two studied movement tasks (tongue and hand)? Our initial hypothesis is that motor execution should active a larger number of areas in the brain, since it has to (a) decide on precise movement to execute and (b) execute it by recruiting the spinal cord. Is this hypothesis backed by the data? In general, what are the brain mechanisms in play here?  

For (2), our goal is to decode the hand movement and tongue movement with an efficient model. Can we achieve a high decoding accuracy (above 90%)? What is the minimum number of parameters required for such decoding (best trade-off number of parameters vs efficiency)? Can we build an efficient decoding algorithm suitable for a real-time usage? Can we predict the motor task prior to its onset and, if so, by how much time?  

We will attempt to tackle these questions by analysis the dataset: preprocessing, extracting features (Power Spectrum Density, ERP, electrode position) and building suitable models.

# Data analysis

## Utils 

In [ ]:
#@title Utils

import os, requests
import numpy as np
from matplotlib import rcParams
from matplotlib import pyplot as plt
from scipy import signal
!pip install mne
import mne
import numpy as np
import scipy

rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

alldat = ''
dataLoaded = False
fname = 'motor_imagery.npz'

def _fetch_data():
    ''' Download data '''

    url = "https://osf.io/ksqv8/download"
    if not os.path.isfile(fname):
        try:
            r = requests.get(url)
        except requests.ConnectionError:
            print("!!! Failed to download data !!!")
        else:
            if r.status_code != requests.codes.ok:
                print("!!! Failed to download data !!!")
            else:
                with open(fname, "wb") as fid:
                    fid.write(r.content)

def get_data(subject, experiment):
    ''' Return the data

        Input:
          subject: subject id
          expeirment: experiment number (0 = real; 1 = imaginary)
    '''
    global dataLoaded, alldat
    _fetch_data()

    if dataLoaded == False:
      alldat = np.load(fname, allow_pickle=True)['dat']
      dataLoaded = True

    dat = alldat[subject][experiment]
    # convert it to float32 for numerical ops
    V = dat["V"].astype("float32")
    dat["V"] = V

    return dat

def preprocess_voltage(V, filter_type):
    ''' Filter voltage

        Args:
            V: voltage
            filter_type: 'high' | 'low'

        Returns: filtered voltage
    '''
    # high-pass filter above 50 Hz
    # remove the noise
    b, a = signal.butter(3, [50], btype=filter_type, fs=1000)
    V = signal.filtfilt(b, a, V, 0)

    # compute smooth envelope of this signal = approx power
    V = np.abs(V) ** 2
    b, a = signal.butter(3, [10], btype="low", fs=1000)
    V = signal.filtfilt(b, a, V, 0)

    # normalize each channel so its mean power is 1
    V = V / V.mean(0)

    return V

def plot_electrodes(dat, filter_type, electrodes):
  ''' Plot given electrodes

      Args:
          dat: data
          filter_type: pre-processing filter (high|low)
          electrodes: set of electrodes to plot
  '''

  V_high = preprocess_voltage(dat['V'], filter_type)

  # average the broadband power across all tongue and hand trial
  nt, nchan = V_high.shape
  nstim = len(dat['t_on'])

  # TODO why 2000 here is enough for all trials
  #trange = np.arange(0, 2000)
  BEGIN = 0 # 2000 #0
  END = 2000 # 4000
  trange = np.arange(BEGIN, END)
  ts = dat['t_on'][BEGIN:,np.newaxis] + trange
  V_epochs = np.reshape(V_high[ts, :], (nstim, END, nchan))

  V_tongue = (V_epochs[dat['stim_id']==11]).mean(0)
  V_hand   = (V_epochs[dat['stim_id']==12]).mean(0)

  # let's find the electrodes that distinguish tongue from hand movements
  # note the behaviors happen some time after the visual cue
  from matplotlib import pyplot as plt

  plt.figure(figsize=(20,10))
  #for j in range(46):
  for j in electrodes:
    ax = plt.subplot(5,10,j+1)
    plt.plot(trange, V_tongue[:,j])
    plt.plot(trange, V_hand[:,j])
    plt.title('ch%d'%j)
    plt.xticks([0, 1000, 2000])
    plt.ylim([0, 4])


def get_mne_structs(dat, epoch, filter = True):
  '''
    Return mne structs (raw data and epoched data)
    Args:
      dat: original numpy data
      epoch: (epoch_begin, epoch_end) [time in seconds]
  '''

  V = dat['V']
  locs = dat['locs'] / 1000 # 1000: mm -> m
  n_channels = dat['V'].shape[1]
  srate = dat['srate']
  stim_id = dat['stim_id']
  t_on = dat['t_on']

  elec_indexes_str = [str(ch) for ch in range(n_channels)]
  locs_dict = dict(zip(elec_indexes_str, locs))
  montage = mne.channels.make_dig_montage(locs_dict, coord_frame='mni_tal')

  info = mne.create_info(n_channels, sfreq=srate, ch_types='ecog')
  raw = mne.io.RawArray(V.T, info)
  # filtering the raw data
  if (filter): 
    _ = raw.filter(l_freq=76, h_freq=110)
  raw.set_montage(montage)

  events = np.vstack((t_on.astype(int), np.zeros(t_on.size).astype(int)))
  events = np.vstack((events, stim_id.astype(int))).T
  event_dict = {'tongue': 11, 'hand': 12}
  epochs = mne.Epochs(raw, events, event_id = event_dict,
                      tmin = epoch[0], tmax = epoch[1], baseline=None)
  return raw, epochs

def plot_fft(dat, channel, num_samples):
    '''
        Plot time vs amplitude & frequency vs power
    '''
    V = dat['V'][:,channel][0:num_samples]
    sampling_rate = dat['srate']
    window_length = 51
    polyorder = 3

    plt.figure(figsize=(12, 12))

    plot_rows = 2; plot_cols = 1; plot_idx = 1

    plt.subplot(plot_rows, plot_cols, plot_idx); plot_idx = plot_idx + 1
    # we see effect of zscoring
    plt.plot(signal.savgol_filter(V, window_length, polyorder))

    plt.xlabel("time")
    plt.ylabel("amplitude")
    plt.xlim(0,num_samples)

    V_fft = scipy.fft.fft(V)
    N = len(V)
    n = np.arange(N)  # S (samples)
    T = N / sampling_rate  # [S/(S/s)] = s
    freq = n / T  # [S/s] = [Hz](V)

    logpower = signal.savgol_filter(
        np.log(np.abs(V_fft) ** 2), window_length, polyorder)

    plt.subplot(plot_rows, plot_cols, plot_idx); plot_idx = plot_idx + 1
    plt.plot(freq, logpower)
    plt.xlabel('Freq (Hz)')
    plt.ylabel('FFT logpower')
    plt.xlim(0,150)


## Activation plot in time

We'll plot the activation for all electrodes. First we define to functions to preprocess/filter and then a plotting function. 

* Only patient 0 shows a very distinct activation w/ the provide filter parameters
    * as per original notebook, we see activation on electrode 20 for hand and 42 for tongue
    * hand = orange
    * tongue = blue
    * What kind of parameters do work w/ do see some activation for other patients? 
    * What is different about patient 0?

In [ ]:
plot_electrodes(get_data(subject=0,experiment=0), 'high', range(46))
plot_electrodes(get_data(subject=1,experiment=0), 'high', range(46))

In [ ]:
# As low filer, we've more distinction among patients apparently. 

plot_electrodes(get_data(subject=0,experiment=0), 'low', range(46))
plot_electrodes(get_data(subject=1,experiment=0), 'low', range(46))

- Comparing the evoked responses across subjects, we see no meaningful pattern emmerging, though we can see some good enough differents for hand and tongue. 

- Patients 1 and 7 seems to have a particularly noisy data set w/ the analysis and channel combining that was done here (gfp).

- gfp. What is this? 

## Analyzing with MNE

Beyond gamma band [Miller, 2008]. Desynchronization of the gamma band. 
Decoding hand vs tongue could be done w/ a very simple number J0

Time interval for each recording: 
* for analysis they take a slot [1, 2.5s], there is some overlap across trials, there is some persistent activity. After you stopped movement, the signal doesn't stop right away 
* we can some clear distinction in the evoked potential across individuals

In [ ]:
for s in range(0,1):
  for e in range(2):
    dat = get_data(s, e)
    
    raw, epochs = get_mne_structs(dat, (1,2.5), filter=False)
    montage = epochs.get_montage()
    #montage.plot()
    
    evoked_hand = epochs['tongue'].average()
    evoked_tongue = epochs['hand'].average()
    print(evoked_hand, evoked_tongue)
    
    mne.viz.plot_compare_evokeds([evoked_hand, evoked_tongue])

In [ ]:
## Other MNE analysis 
# - fft response 
# - response in time plot

# some patients seem to have a strange montage w/ electrodes messed up

for s in range(7):
    dat = get_data(s, 0)
    
    raw, epochs = get_mne_structs(dat, (1,2.5))
    montage = epochs.get_montage()
    montage.plot()

In [ ]:
# let's do some further pre-processing as saing so far
dat = get_data(0, 0)
raw, epochs = get_mne_structs(dat, (1,2.5))
print(raw)
print(epochs)

events = epochs.events
event_id = epochs.event_id
print(events)


In [ ]:
dat = get_data(0, 0)
raw, epochs = get_mne_structs(dat, (1,2.5))

events = epochs.events
event_id = epochs.event_id

mne.viz.plot_events(events, sfreq=raw.info['sfreq'], 
                    first_samp=raw.first_samp, event_id=event_id)

epochs['hand'].plot_image()
epochs['tongue'].plot_image()
evoked_hand.plot(gfp=True)
evoked_tongue.plot(gfp=True)


In [ ]:
dat = get_data(0, 0)

raw, epochs = get_mne_structs(dat, (1,2.5))

print(raw)

mne.viz.plot_raw_psd(raw, fmin=76, fmax=100)

# Abstract

_**Cross-session decoding accuracy: motor imagery vs. motor execution**_

Brain Computer Interfaces (BCI) aim to restore or augment human skills via efficient neuroprosthetic solutions. 
One common BCI paradigm is motor imagery decoding: a subject is instructed to imagine making a movement without actually executing it.

However, few studies so far focused on motor execution. 
Hence, it is unknown whether decoding algorithms for motor imagery would have a similar accuracy on motor execution. 
On the other hand, prior research has shown that both motor imagery and execution have similar brain activation patterns. 
Therefore, we hypothesize that a model trained on motor imagery data would have similar performance on motor execution data. 

Here we evaluate a traditional BCI pipeline composed of filtering on the high-frequency band (76-100Hz), 
Principal Component Analysis (PCA) for dimensionality reduction and Support Vector Machine (SVM) for classification. 
We then evaluate the cross-session accuracy for each subject. 
We perform such analysis on the data from Miller, 2019, composed of electrocorticography (ECoG) recordings collected from eight patients. 

We aim to show that, if we train a decoding model on motor imagery data, then we can use it to 
decode motor execution data and obtain similar accuracy across these two experiments. 
This would allow us to conclude that, since accuracy is preserved, we can decode ECoG data for both motor execution and imagery tasks using the same model. 
This would also further validate previous studies showing that the brain activation for motor execution and imagery are similar. 

Future research should attempt to replicate these findings on other datasets and, in particular, on electroencephalogram (EEG) data. 




# Data classification

Let's just try some naive classification pipeline and see what happens. 

In [ ]:
# @title Imports 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC  # "Support Vector Classifier"
from mne.decoding import CSP

In [ ]:
# Classification pipeline functions

def get_classifier_data(epochs):
  # Get training data
  X = epochs.get_data() # events X channels X value
  y = epochs.events[:,-1] # flatten out array
  le = LabelEncoder() # label events y between [0,n_class-1]
  y = le.fit_transform(y)

  return X, y

def train_classifier(s, e):
  # Get training data
  dat = get_data(subject = s, experiment = e)
  raw, epochs = get_mne_structs(dat, (0,2)) # filtered data
  X, y = get_classifier_data(epochs)

  # Build pipeline
  csp = CSP(n_components=8, 
            transform_into='average_power',
            reg='ledoit_wolf',
            cov_est='epoch',
            component_order='mutual_info',
            rank='info')
  pipeline = make_pipeline(
      csp, 
      SVC(C=1, kernel='linear'))
  
  # Evaluate pipeline
  acc = cross_val_score(pipeline, X, y, cv=5)

  # Fit TODO why not fit after cross validatation?
  pipeline.fit(X, y)

  return pipeline, acc

def train_classifier_all_subjects(experiment):
  # load training data (T)
  pipelines = []
  accuracies = []

  for s in range(7):
    pipeline, acc = train_classifier(s, experiment)

    pipelines.append(pipeline)
    accuracies.append(acc)

  return pipelines, accuracies

In [ ]:
# Run pipeline for all subjects
pipe_motor, acc_motor = train_classifier_all_subjects(0)
pipe_img, acc_img = train_classifier_all_subjects(1)

In [ ]:
print("Train on motor imagery & validate on motor execution")
print("Subject,TestAccuracy")
for s in range(7):
  # get validation data
  datE = get_data(subject = s, experiment = 0)
  rawE, epochsE = get_mne_structs(datE, (0, 2)) # 70%, 0.1
  X_E, y_E = get_classifier_data(epochsE)

  # score trained model
  print(s, ',', pipe_img[s].score(X_E, y_E) * 100)

# avg accuracy = 75%

In [ ]:
print("Train on motor execution & validate on motor imagery")
print("Subject,TestAccuracy")
for s in range(7):
  # get validation data
  datE = get_data(subject = s, experiment = 1)
  rawE, epochsE = get_mne_structs(datE, (0, 2)) # 70%, 0.1
  X_E, y_E = get_classifier_data(epochsE)

  # score trained model
  print(s, ',', pipe_motor[s].score(X_E, y_E) * 100)

# avg accuracy = 54%